# Mount google drive

 **It is heighly recommended to use google colab and put the kitty dataset on your google drive**

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

# Install dependencies

In [ ]:
# Switch to tensorflow 1
%tensorflow_version 1.x

# Install protobuf
!pip install protobuf protobuf-compiler

# Clone the repo
!git clone https://github.com/tensorflow/models.git

# Compile protos.
%cd models/research
!protoc object_detection/protos/*.proto --python_out=.

# Install TensorFlow Object Detection API.
%cp object_detection/packages/tf1/setup.py .
!python -m pip install .

# Test the installation
!python object_detection/builders/model_builder_tf1_test.py
%cd ../..

# Download Faster RCNN model

In [ ]:
%cd /content
!wget http://download.tensorflow.org/models/object_detection/faster_rcnn_resnet101_kitti_2018_01_28.tar.gz
!tar -xvf faster_rcnn_resnet101_kitti_2018_01_28.tar.gz

# Generate TF record for training

In [ ]:
# Make sure to fix the paths according to your setup.
# Note that kitti_label_map.pbtxt has been included with this notebook
!python /content/models/research/object_detection/dataset_tools/create_kitti_tf_record.py \
--data_dir="/content/gdrive/MyDrive/deep box/data" \
--output_path="/content/gdrive/MyDrive/deep box/output/kitti.record" \
--classes_to_use=car,pedestrian,truck,cyclist \
--label_map_path="/content/gdrive/MyDrive/deep box/kitti_label_map.pbtxt" \
--validation_set_side=500

# Train the model

In [ ]:
%cp models/research/object_detection/legacy/train.py models/research/object_detection
%cp /content/models/research/object_detection/legacy/eval.py /content/models/research/object_detection
%cd /content

# Make sure to fix the paths according to your setup.
!python models/research/object_detection/train.py \
--logdir="/content/gdrive/MyDrive/deep box/log" \
--train_dir="/content/gdrive/MyDrive/deep box/output_train2" \
--pipeline_config_path="/content/gdrive/MyDrive/deep box/pipeline.config"

# Evaluate the model

In [ ]:
%cd /content/models/research

# Make sure to fix the paths according to your setup.
!python object_detection/eval.py \
        --logtostderr \
        --checkpoint_dir="/content/gdrive/MyDrive/deep box/output_train2" \
        --eval_dir="/content/gdrive/MyDrive/deep box/output_eval" \
        --pipeline_config_path="/content/gdrive/MyDrive/deep box/output_train2/pipeline.config"

# Export the model

In [ ]:
%cd /content/models/research/
# Make sure to fix the paths according to your setup.
!python object_detection/export_inference_graph.py \
--input_type=image_tensor \
--pipeline_config_path="/content/gdrive/MyDrive/deep box/output_train1/pipeline.config" \
--trained_checkpoint_prefix="/content/gdrive/MyDrive/deep box/output_train2/model.ckpt-25000" \
--output_directory="/content/gdrive/MyDrive/deep box/frozen_model"